In [ ]:
from pathlib import Path
from dotenv import load_dotenv
import os
from huggingface_hub import login

load_dotenv()
login(token=os.getenv("HF_TOKEN"))

In [ ]:
print(os.getenv("HF_TOKEN"))

In [1]:
import torch
print(torch.__version__)
print(torch.cuda.is_available())
print(torch.version.cuda)
print(torch.cuda.device_count())
print(torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No CUDA")


2.7.1+cpu
False
None
0
No CUDA


# Article text scrapping

## newspaper4k

In [ ]:
import newspaper
article = newspaper.article('https://edition.cnn.com/2023/10/29/sport/nfl-week-8-how-to-watch-spt-intl/index.html')
print(article.text)

['patrick', 'mahomes', 'history', 'nfl', 'week', 'broncos', 'denver', 'p', 'm', '00', 'pittsburgh', 'steelers', 'game', 'win', 'los', 'angeles', 'dallas', 'quarterback', 'pickett', 'chiefs', 'season', 'victory', 'rams', 'cowboys', 'games', 'matchup', 'jacksonville', 'lawrence', 'kansas', '25', 'espn', 'bay', 'fox', 'kenny', 'run']
Jacksonville Jaguars @ Pittsburgh Steelers, 1:00 p.m.
ET on CBS Two potential AFC playoff contenders face off when the Pittsburgh Steelers host the Jacksonville Jaguars on Sunday afternoon.
That’s just what you’re going to get.” Kansas City Chiefs @ Denver Broncos, 4:25 p.m.
ET Kansas City Chiefs @ Denver Broncos, 4:25 p.m.
Australia: NFL+, ESPN, 7Plus Brazil: NFL+, ESPN Canada: NFL+, CTV, TSN, RDS Germany: NFL+, ProSieben MAXX, DAZN Mexico: NFL+, TUDN, ESPN, Fox Sports, Sky Sports UK: NFL+, Sky Sports, ITV, Channel 5 US: NFL+, CBS, NBC, FOX, ESPN, Amazon Prime


# Text summarisation and claim extraction

Datasets:

blender-nlp/NewsClaims
https://github.com/blender-nlp/NewsClaims

General Claim dataset | AI-on-Demand
https://www.ai4europe.eu/research/ai-catalog/general-claim-dataset

Fact Extraction and VERification
https://fever.ai/dataset/fever.html

Babelscape/FENICE: FENICE (Factuality Evaluation of Summarization based on Natural Language Inference and Claim Extraction) is a factuality-oriented metric for summarization.
https://github.com/Babelscape/FENICE



to research:

Tswings/AVeriTeC-DCE
https://github.com/Tswings/AVeriTeC-DCE/tree/main?tab=readme-ov-file

2406.03239
https://arxiv.org/pdf/2406.03239

CheckThat! competition
https://checkthat.gitlab.io/clef2025/task2/

## Ollama Qwen

In [ ]:
from langchain.prompts import PromptTemplate
from langchain_ollama import OllamaLLM
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field


prompt = PromptTemplate.from_template("""

You are an assistant to a fact-checker. You will be given 

Summarize the following text into distinct bullet-pointed claims, according to the following instructions:
- Be concise.
- Make sure each claim is a self-contained statement that can be understood independently; don't use pronouns or references to other claims.
    - For example, instead of "This crisis is real," write "The global warming crisis is real."
    - Instead of "This hysteria might lead to restrictions," write "Global warming hysteria might lead to restrictions."
    - Instead of "It is pretentious to think we can destroy God's creation," write "It is pretentious to think humans can destroy God's creation."
    - Instead of "We are becoming environmental hypochondriacs," write "Society is becoming environmental hypochondriacs."
    - Instead of "This leads to...," write "Ecological hysteria leads to..."; be specific about the subject of the claim.
    - Avoid vague terms like "this" or "it" without clear antecedents.
    - Be specific about what you are referring to in each claim.
- Make sure every statement is unique and not a repetition of another claim in different words.
- Don't begin your response with "Here are the claims" or similar phrases; just list the claims directly.
    - Do not include any additional commentary or explanations.
- Don't start your bullet points with "The speaker says" or "The speaker suggests" or similar phrases; just list the claims directly.

Text:
```
{text}
```

{format_instructions}
""")
llm = OllamaLLM(model="qwen:7b-chat", temperature=0, max_tokens=1024)

class Claims(BaseModel):
    claims: list[str] = Field(description="List of claims extracted from the text.")

parser = PydanticOutputParser(pydantic_object=Claims)

chain = prompt | llm | parser

def extract_claims(text: str) -> list[str]:
    return chain.invoke({"text": text, "format_instructions": parser.get_format_instructions()}).claims

In [3]:
print(extract_claims("""What I’m suggesting is we have a sort of an eco-evangelical hysteria going on and it leads me to almost wonder if we are becoming a nation of environmental hypochondriacs that are willing to use the power of the state to impose enormous restrictions on the rights and the comforts of, and incomes of individuals who serve essentially a paranoia, a phobia, that has very little fact evidence in fact. Now these are observations that are popular to make because right now it's almost taken as an article of faith that this crisis is real. Let me say I take it as an article of faith if the Lord God almighty made the heavens and the Earth, and He made them to His satisfaction and it is quite pretentious of we little weaklings here on earth to think that, that we are going to destroy God’s creation."""))

OutputParserException: Invalid json output: Here are the bullet-pointed claims based on the given text:

1. Ecological hysteria is present.
2. The phenomenon is described as an eco-evangelical hysteria.
3. This hysteria leads to a questioning of societal attitudes.
4. Society is being compared to environmental hypochondriacs.
5. These individuals are perceived as willing to use state power for restrictions.
6. The crisis surrounding this issue is considered real by some.
7. The speaker takes it as an article of faith that the crisis is real, if God made the heavens and Earth.

Each claim is self-contained and can be understood independently without referring to other claims.
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/OUTPUT_PARSING_FAILURE 

## T5

In [ ]:
from transformers import T5ForConditionalGeneration, T5Tokenizer

tokenizer = T5Tokenizer.from_pretrained("Babelscape/t5-base-summarization-claim-extractor")
model = T5ForConditionalGeneration.from_pretrained("Babelscape/t5-base-summarization-claim-extractor")
summary = 'Simone Biles made a triumphant return to the Olympic stage at the Paris 2024 Games, competing in the women’s gymnastics qualifications. Overcoming a previous struggle with the “twisties” that led to her withdrawal from events at the Tokyo 2020 Olympics, Biles dazzled with strong performances on all apparatus, helping the U.S. team secure a commanding lead in the qualifications. Her routines showcased her resilience and skill, drawing enthusiastic support from a star-studded audience'

tok_input = tokenizer.batch_encode_plus([summary], return_tensors="pt", padding=True)
claims = model.generate(**tok_input)
claims = tokenizer.batch_decode(claims, skip_special_tokens=True)
print(claims)

c:\Users\arkma\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
print(claims)

["Simone Biles made a triumphant return to the Olympic stage at the Paris 2024 Games. Biles competed in the women’s gymnastics qualifications. Biles overcame a previous struggle with the 'twisties' that led to her withdrawal from events at the Tokyo 2020 Olympics. Biles dazzled with strong performances on all apparatus. The U.S. team secured a commanding lead in the qualifications. Her routines showcased her resilience and skill. The routines attracted enthusiastic support from a star-studded audience."]


## GemmaAPS

https://huggingface.co/google/gemma-7b-aps-it

https://huggingface.co/sheldonrobinson/gemma-7b-aps-it-Q4_K_M-GGUF/tree/main

In [ ]:
import nltk
import re

nltk.download('punkt')

start_marker = '<s>'
end_marker = '</s>'
separator = '\n'

def create_propositions_input(text: str) -> str:
    input_sents = nltk.tokenize.sent_tokenize(text)
    propositions_input = ''
    for sent in input_sents:
        propositions_input += f'{start_marker} ' + sent + f' {end_marker}{separator}'
    propositions_input = propositions_input.strip(f'{separator}')
    return propositions_input

def process_propositions_output(text):
    pattern = re.compile(f'{re.escape(start_marker)}(.*?){re.escape(end_marker)}', re.DOTALL)
    output_grouped_strs = re.findall(pattern, text)
    predicted_grouped_propositions = []
    for grouped_str in output_grouped_strs:
        grouped_str = grouped_str.strip(separator)
        props = [x[2:] for x in grouped_str.split(separator)]
        predicted_grouped_propositions.append(props)
    return predicted_grouped_propositions


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\arkma\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model_id = 'google/gemma-7b-aps-it'
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map='auto',
    torch_dtype=torch.bfloat16,
)

passage = "For more than 40 years, the lyrics of American Pie have been puzzled over. This week the handwritten lyrics sold for more than $1 million at auction. The verses contain hidden references to seminal events of the 50s and 60s. It includes nods to Buddy Holly, Charles Manson and Martin Luther King."
messages = [{'role': 'user', 'content': create_propositions_input(passage)}]
inputs = tokenizer.apply_chat_template(messages, return_tensors='pt', add_generation_prompt=True, return_dict=True).to(model.device)

output = model.generate(**inputs, max_new_tokens=4096, do_sample=False)
generated_text = tokenizer.batch_decode(output[:, inputs['input_ids'].shape[1]:], skip_special_tokens=True)[0]
result = process_propositions_output(generated_text)
print(result)


Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]Error while downloading from https://cdn-lfs-us-1.hf.co/repos/39/77/397734a2bee7d00422ac575da0dd0cf0e1ad9156aaf41676efa481dd6e6bdfb8/fd1b87c972b59151d0f14f161714f530db453b46c288e08adff53c99c1a320f4?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27model-00001-of-00004.safetensors%3B+filename%3D%22model-00001-of-00004.safetensors%22%3B&Expires=1750059177&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTc1MDA1OTE3N319LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmhmLmNvL3JlcG9zLzM5Lzc3LzM5NzczNGEyYmVlN2QwMDQyMmFjNTc1ZGEwZGQwY2YwZTFhZDkxNTZhYWY0MTY3NmVmYTQ4MWRkNmU2YmRmYjgvZmQxYjg3Yzk3MmI1OTE1MWQwZjE0ZjE2MTcxNGY1MzBkYjQ1M2I0NmMyODhlMDhhZGZmNTNjOTljMWEzMjBmND9yZXNwb25zZS1jb250ZW50LWRpc3Bvc2l0aW9uPSoifV19&Signature=EYTUAjdDc0gniDhC24fHjWrNmkxAVRl4keL4VDm87UB-DHYc4KKe2zDVu1s86yUMw5%7EkhEoeSGFmfcWY2457zv%7EnTKL6zmi0iybDTlLOrazSm8E9FTSd869K-b2G6p%7Ee72rg4NPENQaPOw57REnf6ovN739I4OuC0rsN3XHFL

KeyboardInterrupt: 

## Mistral fine-tune

https://huggingface.co/SAGESSE-EPFL/Mistral-7b-claims-extraction

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import torch

model_id = "SAGESSE-EPFL/Mistral-7b-claims-extraction"
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float16,     # or bfloat16
    device_map="auto",             # auto-assign to GPU/CPU
)

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=256,
)

submission = "Submission text here."
parent_comment = "Parent comment here."
input_comment = "This is the Reddit comment from which you want to extract claims."

# same prompt template shown in the model card
prompt = f"""### Instruction:
You are a helpful assistant for claim extraction from Reddit comments. A claim is a statement that expresses a point of view or a conclusion that a speaker believes is true...
### Context:
Submission:
{submission}

Parent comment:
{parent_comment}

### Comment:
{input_comment}

### Response:
"""

output = pipe(prompt)
response_text = output[0]["generated_text"]

import json
claims = json.loads(response_text.split("```")[-1])["claims"]
print(claims)


c:\Users\arkma\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Fetching 6 files:   0%|          | 0/6 [00:00<?, ?it/s]Error while downloading from https://cdn-lfs-us-1.hf.co/repos/15/a0/15a079b835e657790743d8acb363532e5b14cbda61d54bd1f8d127d9193e7fe1/b2e3f1ac0223480ff668df617915c60357c894e0c094c75fe6864ca060b7cdf2?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27model-00001-of-00006.safetensors%3B+filename%3D%22model-00001-of-00006.safetensors%22%3B&Expires=1750075970&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTc1MDA3NTk3MH19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmhmLmNvL3JlcG9zLzE1L2EwLzE1YTA3OWI4MzVlNjU3NzkwNzQzZDhhY2IzNjM1MzJlNWIxNGNiZGE2MWQ1NGJkMWY4ZDEyN2Q5MTkzZTdmZTEvYjJlM2YxYWMwMjIzNDgwZmY2NjhkZjYxNzkxNWM2

: 

: 

## Claimify (custom implementation)

https://arxiv.org/pdf/2502.10855

check claimify.ipynb for a partial implementation (failed due to underpowered models)

# Claim decontextualisation

chrF (character-level F score): Measures the similarity between decontextualized claims and gold standard claims.

AVeriTeC-DCE: A dataset derived from AVeriTeC containing decontextualized claims for document-level claim extraction.

CLAN: A comprehensive dataset comprising more than 6,000 instances of social media posts alongside their respective normalized claims.

FEVERFact: Contains 17,000 atomic factual claims extracted from 4,000 contextualized Wikipedia sentences

In [ ]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_ollama import ChatOllama

llm = ChatOllama(model="qwen:7b-chat", temperature=0)

decontextualisation_prompt = PromptTemplate.from_template(r"""Perform claim decontextualisation given context.
You will return the output of a function named decontextualise(context:str, claim:str)->str.
The return value must be a decontextualised version of the input string.

For example:
decontextualise(context='Simone Biles made a triumphant return to the Olympic stage at the Paris 2024 Games. Biles competed in the women’s gymnastics qualifications.', input='The U.S. team secured a commanding lead in the qualifications.') should output 'The U.S. team secured a commanding lead in the women's gymnastics qualifications at the 2024 Paris Games.'

Your output will be passed on to a Python interpreter, SO NO OTHER WORDS!

Return the output of the following function call:
decontextualise(context={context}, claim={claim})
"""
)

str_parser = StrOutputParser()

chain = decontextualisation_prompt | llm | str_parser

context = "NASA's Artemis program aims to return humans to the Moon by 2025, including the first woman and person of color. The mission will use the Space Launch System rocket and the Orion spacecraft."
claim = "The spacecraft is designed to support long-duration missions beyond low Earth orbit."

response = chain.invoke({"context": context, "claim": claim})
print(response)


Output: 'The spacecraft is designed to support long-duration missions beyond low Earth orbit as part of NASA's Artemis program, aiming to return humans to the Moon by 2025.'


# Coreference resolution

| Model                       | F1 Score (OntoNotes) | Parameters      | Key Advantage              | Speed Comparison         |
|-----------------------------|----------------------|-----------------|----------------------------|--------------------------|
| Maverick                    | 83.6                 | 500M            | Best accuracy + efficiency | 170x faster inference    |
| LingMess                    | 81.4                 | ~340M           | Linguistic categorization  | Standard speed           |
| F-coref                     | 78.5                 | ~148M (distilled)| Ultra-fast processing      | 28x faster               |
| SpanBERT-large + e2e        | 79.6                 | ~340M           | Strong baseline            | Standard speed           |
| s2e + Longformer-Large      | 80.3                 | ~340M           | Memory efficient           | Improved speed           |


https://github.com/shon-otmazgin/fastcoref?tab=readme-ov-file

https://github.com/SapienzaNLP/maverick-coref/tree/main

In [1]:
%pip install neuralcoref

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'error'
Note: you may need to restart the kernel to use updated packages.


  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> [23 lines of output]
      Traceback (most recent call last):
        File "c:\Users\arkma\AppData\Local\Programs\Python\Python313\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 389, in <module>
          main()
          ~~~~^^
        File "c:\Users\arkma\AppData\Local\Programs\Python\Python313\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 373, in main
          json_out["return_val"] = hook(**hook_input["kwargs"])
                                   ~~~~^^^^^^^^^^^^^^^^^^^^^^^^
        File "c:\Users\arkma\AppData\Local\Programs\Python\Python313\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 143, in get_requires_for_build_wheel
          return hook(config_settings)
        File "C:\Users\arkma\AppData\Local\Temp\pip-build-env-3j7msfyd\overlay\Lib\site-packa

# Verifiability/checkworthy/falsifiability/objectivity check

https://checkthat.gitlab.io/clef2025/task1/

In [ ]:
# Verifiability ("checkworthy") checks
verifiability_prompt = PromptTemplate.from_template(r"""
You are an assistant to a fact-checker.
You will be given a single sentence and you will determine if the claim is
1. a verifiable, checkworthy, factual and falsifiable statement that can be proven true or false using evidence or citations found online,
2. an opinion,
3. a personal statement/fact or belief, 
4. or not a claim at all (like a question or an imperative statement).

You will return the output of a function named is_checkworthy(claim:str)->str.
The return value must be one of the following:
- "checkworthy": if the claim is a verifiable, checkworthy, factual and falsifiable statement that can be proven true or false using evidence or citations found online.
""")
chain = verifiability_prompt | llm | str_parser
response = chain.invoke({"claim": "The Earth is flat."})
print(response)

unverifiable


In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification

model = BertForSequenceClassification.from_pretrained("yevhenkost/claimbuster-yesno-binary-bert-base-cased")
tokenizer = BertTokenizer.from_pretrained("yevhenkost/claimbuster-yesno-binary-bert-base-cased")

text_inputs = ["The Earth is flat.", "The sky is blue.", "I can fit my entire foot in my mouth.", "Pizza tastes better with pineapple."]
model_inputs = tokenizer(text_inputs, return_tensors="pt")
model_output = model(**model_inputs)

c:\Users\arkma\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


OSError: There was a specific connection error when trying to load yevhenkost/claimbuster-yesno-binary-bert-base-cased:
401 Client Error: Unauthorized for url: https://huggingface.co/yevhenkost/claimbuster-yesno-binary-bert-base-cased/resolve/main/config.json (Request ID: Root=1-685a4c9b-1f36eed77258ca634f5d1bef;f7bf5354-2e6b-494f-b8b1-f10065e05cb2)

Invalid credentials in Authorization header

# Web searching

In [ ]:
import serpapi
import pickle

import os
from dotenv import load_dotenv
load_dotenv()
API_KEY: str = os.getenv("SERPAPI_KEY", "")

TARGET_SITES = pickle.load(open("reliable_list.pkl", "rb"))

query = "Climate change is a hoax."

params = {
    "engine": "google_light",
    "q": query,
    "api_key": API_KEY,
}

search = serpapi.search(params)
print(search)

filtered_urls = []
for result in search.get("organic_results", []):
    url = result.get("link")
    if any(site in url for site in TARGET_SITES):
        filtered_urls.append(url)

print("Filtered URLs:")
for url in filtered_urls:
    print(url)

# Claim verification

## Teapot LLM

https://huggingface.co/teapotai/teapotllm

small, locally runable, answers questions and apparently hallucination resistant

Can questions can be framed as "Is this claim true? Claim: " for the purposes of this project?

In [1]:
from teapotai import TeapotAI

# Sample context
context = """
The Eiffel Tower is a wrought iron lattice tower in Paris, France. It was designed by Gustave Eiffel and completed in 1889.
It stands at a height of 330 meters and is one of the most recognizable structures in the world.
"""

teapot_ai = TeapotAI()

answer = teapot_ai.query(
    query="Is this statement supported: The height of the Eiffel Tower is 320 meters.", 
    context=context
)
print(answer) # => "The Eiffel Tower stands at a height of 330 meters. "


c:\Users\arkma\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\arkma\AppData\Local\Programs\Python\Python313\Lib\site-packages\teapotai\teapotai.py:18: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
c:\Users\arkma\AppData\Local\Programs\Python\Python313\Lib\site-packages\pydantic\_internal\_fields.py:198: UserWarning: Field name "schema" in "TeapotTool" shadows an attribute in parent "BaseModel"
  warnings.warn(


 _____                      _         _    ___        __o__    _;;
|_   _|__  __ _ _ __   ___ | |_      / \  |_ _|   __ /-___-\__/ /
  | |/ _ \/ _` | '_ \ / _ \| __|    / _ \  | |   (  |       |__/
  | |  __/ (_| | |_) | (_) | |_    / ___ \ | |    \_|~~~~~~~|
  |_|\___|\__,_| .__/ \___/ \__/  /_/   \_\___|      \_____/
               |_|   
Loading Model


c:\Users\arkma\AppData\Local\Programs\Python\Python313\Lib\site-packages\teapotai\teapotai.py:87: SyntaxWarning: invalid escape sequence '\ '
  |_   _|__  __ _ _ __   ___ | |_      / \  |_ _|   __ /-___-\__/ /


OSError: There was a specific connection error when trying to load teapotai/teapotllm:
401 Client Error: Unauthorized for url: https://huggingface.co/teapotai/teapotllm/resolve/699ab39cbf586674806354e92fbd6179f9a95f4a/config.json (Request ID: Root=1-68596771-4296ee02136e168b4609fec1;4f8d4532-4c46-4279-943b-a9242b117d93)

Invalid credentials in Authorization header

## Roberta-large-mnli

In [2]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import torch.nn.functional as F

model_name = "roberta-large-mnli"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

paragraph = "The government has taken several steps to reduce carbon emissions in recent years."
statement = "The government is not doing anything about carbon emissions."

inputs = tokenizer.encode_plus(statement, paragraph, return_tensors="pt", truncation=True)

with torch.no_grad():
    logits = model(**inputs).logits
    probs = F.softmax(logits, dim=-1)

labels = ["contradiction", "neutral", "entailment"]
prediction = labels[probs.argmax().item()]
print(f"Prediction: {prediction}")

Prediction: contradiction


# Other

https://checkthat.gitlab.io/clef2025/task3/

https://checkthat.gitlab.io/clef2025/task4/

Another implementation:
https://github.com/mbzuai-nlp/fire